In [1]:
!pip install alpha_vantage

In [2]:
!pip install holiday

In [0]:
MY_API_KEY = 'J4PS1W9LI8IL0E97'
ENDDATE = '2019-10-01'

from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
from datetime import timedelta

In [0]:
ONEYEARDAYS = 252
FREQ = 90

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('ggplot')
import os
import scipy
from scipy import optimize
from scipy.optimize import minimize

In [262]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ts = TimeSeries(key='MY_API_KEY', output_format='pandas')
def get_daily_close(symbol):
    full_data, meta_data = ts.get_daily(symbol = symbol, outputsize='full')
    series_index = pd.Series(full_data.index)
    full_data.index = series_index.apply(lambda t: np.datetime64(t).astype(datetime).date()).values
    assert '4. close' in full_data.columns
    close = full_data['4. close']
    close.index.name = 'Date'
    close.name = symbol
    assert close.index[0] < datetime(2006,1,3).date()
    return close[datetime(2006,1,3).date() : ]

In [0]:
SPY = get_daily_close('SPY')
TLT = get_daily_close('TLT')
GLD = get_daily_close('GLD')
assert len(SPY) == len(TLT)
assert len(TLT) == len(GLD)
m1 = pd.merge(left = SPY, left_index = True, right = TLT, right_index=True, how = 'inner')
dataall = pd.merge(left = m1, left_index = True, right = GLD, right_index=True, how = 'inner')

In [0]:
returns = dataall.pct_change()
returns.dropna(inplace = True)
returns_mul_factor = returns + 1
## Calculating covariance matrix
cov_series = returns.rolling(window = FREQ).cov() * ONEYEARDAYS

In [0]:
spy_ret = returns['SPY']
tlt_ret = returns['TLT']
gld_ret = returns['GLD']
day_counts = len(spy_ret)
calendar = returns.index.values

In [0]:
all_cov = []
for i in range(1,day_counts//FREQ + 1):
    ## Indexing the covariance matrix, in pair of (date, covariance matrix)
    all_cov.append( (cov_series.index[(i*FREQ-1)*3][0] ,cov_series.iloc[(i*FREQ-1)*3 : i*FREQ*3]))

In [0]:
def equal_risk(weight, cov):
    diff1 = weight[0]**2 * cov.iloc[0, 0] + \
        weight[0]*weight[1] * cov.iloc[1,0] + \
        weight[0]*(1 - weight[0] - weight[1]) * cov.iloc[2,0]
        
    diff2 = weight[0]*weight[1] * cov.iloc[0,1] + \
        weight[1]**2 * cov.iloc[1,1] + \
        weight[1]*(1 - weight[0] - weight[1]) * cov.iloc[2,1]
        
    diff3 = weight[0]*(1 - weight[0] - weight[1]) * cov.iloc[0,2] + \
        weight[1]*(1 - weight[0] - weight[1]) * cov.iloc[1,2] + \
        (1 - weight[0] - weight[1])**2 * cov.iloc[2,2]
        
    return np.square(diff1 - diff2)+ np.square(diff1 - diff3) + np.square(diff2 - diff3)

def weight_calculation(cov_df):
    ans = minimize(equal_risk, [0.0, 0.0], (cov_df), method='L-BFGS-B',  bounds=((0,1), (0,1)))
    result = []
    result.extend(list(ans.x))
    result.append(1- ans.x.sum())
    return np.array(result)

In [0]:
rebalance_weight = []
rebalance_date = []
rebalance_pair = {}
for i in range(day_counts//FREQ ):
    res = weight_calculation(all_cov[i][1])
    rebalance_date.append(all_cov[i][0])
    rebalance_weight.append(res)  
    rebalance_pair.update({all_cov[i][0] : res})
rebalance_pair = pd.Series(rebalance_pair)

In [0]:
new_calendar = calendar[calendar > rebalance_date[0]]

In [271]:
rebalance_pair

2006-05-12    [0.3756643717440895, 0.4794101499727612, 0.144...
2006-09-20    [0.306691511675375, 0.5598932778181815, 0.1334...
2007-01-31    [0.40618712130486684, 0.40496361799013614, 0.1...
2007-06-11    [0.2695585832868849, 0.558807587214823, 0.1716...
2007-10-17    [0.2577202026338392, 0.48041343588719604, 0.26...
2008-02-27    [0.2977705531427226, 0.5005439045978101, 0.201...
2008-07-07    [0.3549735116926285, 0.4738846924283521, 0.171...
2008-11-11    [0.2003312098803573, 0.6084778184360065, 0.191...
2009-03-24    [0.24602049611869936, 0.48998480105347103, 0.2...
2009-07-31    [0.29250022015374916, 0.379008807925599, 0.328...
2009-12-08    [0.31455332984068035, 0.4318884434265012, 0.25...
2010-04-20    [0.32526833028270763, 0.4849483767044858, 0.18...
2010-08-26    [0.325369694924215, 0.440872579047761, 0.23375...
2011-01-04    [0.4549419797305526, 0.316437527420834, 0.2286...
2011-05-13    [0.3574901788508035, 0.39268211006223547, 0.24...
2011-09-21    [0.38025469549196184, 0.34

In [272]:
rebalance_weight[0]

array([0.37566437, 0.47941015, 0.14492548])

In [0]:
current_asset = rebalance_weight[0] * 1000
backtest = {rebalance_date[0] : 1000}
for date in new_calendar:
 
    if date not in rebalance_date:
        current_asset = current_asset * (returns_mul_factor.loc[date].values)
        current_level = current_asset.sum()
        backtest.update({date: current_level})
    else:
        current_asset = current_asset * (returns_mul_factor.loc[date].values)
        current_level = current_asset.sum()
        current_asset = rebalance_pair[date] * current_level
        backtest.update({date: current_level})

In [0]:
res = pd.Series(backtest)

In [275]:
res

2006-05-12    1000.000000
2006-05-15     995.109818
2006-05-16    1000.599155
2006-05-17     989.495004
2006-05-18     992.783198
2006-05-19     992.815297
2006-05-22     990.469294
2006-05-23     989.589600
2006-05-24     988.000735
2006-05-25     991.925223
2006-05-26     994.281665
2006-05-30     987.442704
2006-05-31     986.847706
2006-06-01     987.222864
2006-06-02     996.187697
2006-06-05     989.657082
2006-06-06     989.568251
2006-06-07     986.372683
2006-06-08     984.247295
2006-06-09     984.757457
2006-06-12     980.354494
2006-06-13     968.141655
2006-06-14     965.303292
2006-06-15     973.540850
2006-06-16     968.899474
2006-06-19     962.664988
2006-06-20     964.989236
2006-06-21     969.878320
2006-06-22     964.272736
2006-06-23     963.758336
                 ...     
2019-08-26    2491.015555
2019-08-27    2507.871678
2019-08-28    2514.095558
2019-08-29    2518.217310
2019-08-30    2516.581990
2019-09-03    2518.532226
2019-09-04    2534.410964
2019-09-05  